In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [92.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.laun

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]").appName("RDD").getOrCreate()
spark

In [ ]:
df = spark.read.csv("/content/Linear_regression_dataset.csv", header = True, inferSchema = True)
df.show(5)


+-----+-----+-----+-----+-----+------+
|var_1|var_2|var_3|var_4|var_5|output|
+-----+-----+-----+-----+-----+------+
|  734|  688|   81|0.328|0.259| 0.418|
|  700|  600|   94| 0.32|0.247| 0.389|
|  712|  705|   93|0.311|0.247| 0.417|
|  734|  806|   69|0.315| 0.26| 0.415|
|  613|  759|   61|0.302| 0.24| 0.378|
+-----+-----+-----+-----+-----+------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import corr
df.select(corr('var_1', 'output')).show()
df.select(corr('var_1', 'var_2')).show()
df.select(corr('var_2', 'output')).show()

+-------------------+
|corr(var_1, output)|
+-------------------+
| 0.9187399607627283|
+-------------------+

+------------------+
|corr(var_1, var_2)|
+------------------+
|0.3801386662491575|
+------------------+

+-------------------+
|corr(var_2, output)|
+-------------------+
|0.43652698913681093|
+-------------------+



In [ ]:
from pyspark.ml.feature import VectorAssembler
vec_assembler = VectorAssembler(inputCols= ['var_1', 'var_2', 'var_3', 'var_4', 'var_5'], outputCol="features")
features_df = vec_assembler.transform(df)
features_df.show(5, truncate = False)

+-----+-----+-----+-----+-----+------+------------------------------+
|var_1|var_2|var_3|var_4|var_5|output|features                      |
+-----+-----+-----+-----+-----+------+------------------------------+
|734  |688  |81   |0.328|0.259|0.418 |[734.0,688.0,81.0,0.328,0.259]|
|700  |600  |94   |0.32 |0.247|0.389 |[700.0,600.0,94.0,0.32,0.247] |
|712  |705  |93   |0.311|0.247|0.417 |[712.0,705.0,93.0,0.311,0.247]|
|734  |806  |69   |0.315|0.26 |0.415 |[734.0,806.0,69.0,0.315,0.26] |
|613  |759  |61   |0.302|0.24 |0.378 |[613.0,759.0,61.0,0.302,0.24] |
+-----+-----+-----+-----+-----+------+------------------------------+
only showing top 5 rows



In [ ]:
model_df = features_df.select('features', 'output')
model_df.show(5, truncate= False)

+------------------------------+------+
|features                      |output|
+------------------------------+------+
|[734.0,688.0,81.0,0.328,0.259]|0.418 |
|[700.0,600.0,94.0,0.32,0.247] |0.389 |
|[712.0,705.0,93.0,0.311,0.247]|0.417 |
|[734.0,806.0,69.0,0.315,0.26] |0.415 |
|[613.0,759.0,61.0,0.302,0.24] |0.378 |
+------------------------------+------+
only showing top 5 rows



In [ ]:
train_df, test_df = model_df.randomSplit([.8, .2])
print(train_df.count(), len(train_df.columns))

986 2


In [ ]:
train_df.describe().show()

+-------+-------------------+
|summary|             output|
+-------+-------------------+
|  count|                986|
|   mean| 0.3977505070993903|
| stddev|0.03323441774102926|
|    min|              0.301|
|    max|              0.491|
+-------+-------------------+



In [ ]:
test_df.describe().show()

+-------+--------------------+
|summary|              output|
+-------+--------------------+
|  count|                 246|
|   mean|  0.3957032520325204|
| stddev|0.033414364343544896|
|    min|               0.311|
|    max|               0.472|
+-------+--------------------+



In [ ]:
from pyspark.ml.regression import LinearRegression
lin_reg = LinearRegression(labelCol= 'output')

In [ ]:
lin1 = lin_reg.fit(train_df)
lin1.intercept
print(lin1.coefficients)

[0.00033000846189627737,5.785495719105231e-05,0.0002443662794354218,-0.630182579262474,0.49522283051524685]


In [ ]:
train_pred = lin1.evaluate(train_df)


In [ ]:
train_pred.meanSquaredError
test_pred = lin1.evaluate(test_df)
test_pred.rootMeanSquaredError
test_pred.meanSquaredError

0.00013858569682293983